# Fit Neg Binomial Auto-regressive model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial AR parameters ($\beta, \rho, \sigma^2$) for 10 iterations
    - Fix $\beta, \rho, \sigma^2$, fit $r$ using Newton for 10 iterations
    - Repeat until convergence

# Block updates

In [23]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices

function get_V(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = vec[i - 1] * ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

# simulation parameters
Random.seed!(12345)
N = 10000            # sample size
n = 5                # observations per subject
p = 3                # number of fixed effects, including intercept
σ2true = 0.1         # true σ2
ρtrue = 0.5          # true ρ
r = 100              # true dispersion parameter in NB model
β_true = ones(p)     # true beta
V = get_V(ρtrue, n)  # true variance
Γ = σ2true * V;      # true Gamma

In [24]:
# now simulate y
Random.seed!(12345)
d = NegativeBinomial()
link = LogLink()
gcs = Vector{NBCopulaARObs{Float64, typeof(d), typeof(link)}}(undef, N)
res = Vector{Float64}(undef, n)
for i in 1:N
    y = Vector{Float64}(undef, n)
#     Random.seed!(12345 * i)
    X = [ones(n) randn(n, p - 1)]
    η = X * β_true
    μ = exp.(η)
    prob = r ./ (μ .+ r)
    vecd = [NegativeBinomial(r, prob[i]) for i in 1:n]
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
#     Random.seed!(12345 * i)
    rand(nonmixed_multivariate_dist, y, res) # this mutates y!!
    gcs[i] = NBCopulaARObs(y, X, d, link)
end
gcm = NBCopulaARModel(gcs);

In [25]:
initialize_model!(gcm)
@show gcm.β
@show gcm.r
@show gcm.σ2
@show gcm.ρ;

Initializing NegBin r to Poisson regression values


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
gcm.β = [0.9917054424183264, 1.0015008766192153, 1.0054067491065022]
gcm.r = [62.08310723968225]
gcm.σ2 = [0.03117491644226131]
gcm.ρ = [1.0]


In [26]:
# fit negative binomial AR model
@time GLMCopula.fit!(gcm, maxBlockIter=30)

Block iter 1 r = 79.54, logl = -96890.65, tol = 96890.64856995121
Block iter 2 r = 90.91, logl = -96878.32, tol = 0.0001272046100484276
Block iter 3 r = 90.36, logl = -96867.98, tol = 0.00010678403845248314
  5.970401 seconds (5.42 M allocations: 815.792 MiB, 6.71% gc time)


-96867.97583533677

In [27]:
println("estimated β = $(gcm.β); true β = $β_true")
println("estimated rho = $(gcm.ρ[1]); true rho = $ρtrue")
println("estimated σ2 = $(gcm.σ2[1]); true σ2 = $σ2true")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = [1.0048728628237904, 0.9947303695556271, 1.0010291473742292]; true β = [1.0, 1.0, 1.0]
estimated rho = 0.6165379267697961; true rho = 0.5
estimated σ2 = 0.05547752554452006; true σ2 = 0.1
estimated r = 91.32158032867999; true r = 100


# Benchmark/profile code

In [191]:
using BenchmarkTools, SpecialFunctions

function update_HΣ_test!(gcm)
    T = Float64
    fill!(gcm.HΣ, 0.0)
    @inbounds for j in 1:gcm.m
        @simd for i in 1:length(gcm.storage_n)
            gcm.hess1[j, i] = gcm.QF[i, j] * gcm.storage_n[i]
            gcm.hess2[j, i] = gcm.TR[i, j] * gcm.storage_n2[i]
        end
    end
    BLAS.gemm!('N', 'T', -T(1), gcm.hess1, gcm.hess1, T(0), gcm.HΣ)
    BLAS.gemm!('N', 'T', T(1), gcm.hess2, gcm.hess2, T(1), gcm.HΣ)
end

update_HΣ_test! (generic function with 1 method)

In [192]:
gc = gcm.data[1]
β = gcm.β
@btime update_HΣ_test!($gcm)

  65.226 μs (0 allocations: 0 bytes)


2×2 Matrix{Float64}:
 23863.7  23863.7
 23863.7  23863.7

# Let's see how to optimally block update 

## 1 iteration each

In [58]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7640471229274333; true β = 3.757872325600888
estimated variance component 1 = 0.010091096783959048; true variance component 1 = 0.1
estimated variance component 2 = 0.010061528161806528; true variance component 2 = 0.5
estimated r = 59.22108970817275; true r = 100


## 5 iteration each

In [45]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.761120679347187; true β = 3.757872325600888
estimated variance component 1 = 0.04200544450800426; true variance component 1 = 0.1
estimated variance component 2 = 0.020655323963911138; true variance component 2 = 0.5
estimated r = 64.2226158070482; true r = 100


## 10 iteration each

In [51]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7577367229152965; true β = 3.757872325600888
estimated variance component 1 = 0.08454686818713725; true variance component 1 = 0.1
estimated variance component 2 = 0.3859156499408988; true variance component 2 = 0.5
estimated r = 96.07852027325903; true r = 100


## 1 IPOPT iter + complete Newton fit

In [65]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.762274167394871; true β = 3.757872325600888
estimated variance component 1 = 0.044468809218469255; true variance component 1 = 0.1
estimated variance component 2 = 0.046354142614646404; true variance component 2 = 0.5
estimated r = 67.38079696512439; true r = 100


## 5 IPOPT iter + complete Newton fit

In [72]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.759537350779348; true β = 3.757872325600888
estimated variance component 1 = 0.05312813376763176; true variance component 1 = 0.1
estimated variance component 2 = 0.07266779547787298; true variance component 2 = 0.5
estimated r = 70.93622181411288; true r = 100


## 10 IPOPT iter + complete Newton fit (this reached max iter count)

In [79]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.757598085312234; true β = 3.757872325600888
estimated variance component 1 = 0.08847643797375318; true variance component 1 = 0.1
estimated variance component 2 = 0.418885780706873; true variance component 2 = 0.5
estimated r = 98.01186301721411; true r = 100


## Copying GLM.jl

In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.456226847936331; true β = 1.4552872326068422
estimated variance component 1 = 0.10133551134062899; true variance component 1 = 0.1
estimated variance component 2 = 0.4371874111348784; true variance component 2 = 0.5
estimated r = 9.69061414063922; true r = 10


In [73]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.758435534322703; true β = 3.757872325600888
estimated variance component 1 = 0.0934850410967605; true variance component 1 = 0.1
estimated variance component 2 = 0.3827355454688751; true variance component 2 = 0.5
estimated r = 94.45211633725886; true r = 100
